<a href="https://colab.research.google.com/github/EricEricEricJin/ECE539-Group-Project/blob/master/pred_series_mlp_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras

import os

Mounted at /content/drive


In [3]:
# Load dataset

DIR_NAME = "/content/drive/My Drive/Colab Notebooks/ECE539_Project/"

def get_series_labels():
  series = []
  labels = []
  for dirname, _, filenames in os.walk(DIR_NAME + "pred_series"):
    if ("singing" in dirname or "speech" in dirname):
      for filename in filenames:
        for row in np.load(os.path.join(dirname, filename)):
          series.append(row)
          labels.append([0, 1] if "singing" in dirname else [1, 0])
  return series, labels

series, labels = get_series_labels()
series_ds = tf.data.Dataset.from_tensor_slices((series, labels))
print("Number of data", len(series_ds))

for _ in series_ds.take(1):
  input_size = len(_[0])
print("Input size", input_size)

Number of data 116
Input size 50


In [4]:
# Train test splittion
batch_size = 8
series_ds_batched = series_ds.batch(batch_size)
train_ds, val_ds = keras.utils.split_dataset(series_ds_batched, left_size=0.8)
# train_ds = train_ds.batch(batch_size)
# val_ds = val_ds.batch(batch_size)

In [5]:
# Develop a three-layer MLP with 50 inputs and 2 outputs
model = keras.models.Sequential()
model.add(keras.layers.Dense(50, input_dim=input_size, activation="ReLU"))
model.add(keras.layers.Dense(60, activation="ReLU"))
model.add(keras.layers.Dense(50, activation="ReLU"))
model.add(keras.layers.Dense(2, activation="softmax"))
model.compile(loss='binary_crossentropy', optimizer='adam',
            metrics=['accuracy'])

EPOCHS = 50

checkpoint_filepath = f"/content/drive/My Drive/Colab Notebooks/ECE539_Project/pred_series_mlp_weights"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

earlystop_callback = keras.callbacks.EarlyStopping(verbose=1, patience=10)


history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[earlystop_callback, checkpoint_callback],
)

Epoch 1/50
12/12 [==============================] - 3s 163ms/step - loss: 0.9988 - accuracy: 0.5484 - val_loss: 1.9195 - val_accuracy: 0.0000e+00
Epoch 2/50
12/12 [==============================] - 1s 76ms/step - loss: 0.4226 - accuracy: 0.7849 - val_loss: 0.6958 - val_accuracy: 0.6522
Epoch 3/50
12/12 [==============================] - 1s 76ms/step - loss: 0.2907 - accuracy: 0.9462 - val_loss: 0.3889 - val_accuracy: 0.9130
Epoch 4/50
12/12 [==============================] - 0s 4ms/step - loss: 0.2382 - accuracy: 0.9462 - val_loss: 0.3181 - val_accuracy: 0.9130
Epoch 5/50
12/12 [==============================] - 0s 4ms/step - loss: 0.1918 - accuracy: 0.9570 - val_loss: 0.3119 - val_accuracy: 0.9130
Epoch 6/50
12/12 [==============================] - 0s 4ms/step - loss: 0.1547 - accuracy: 0.9785 - val_loss: 0.3088 - val_accuracy: 0.9130
Epoch 7/50
12/12 [==============================] - 0s 4ms/step - loss: 0.1270 - accuracy: 0.9892 - val_loss: 0.3058 - val_accuracy: 0.9130
Epoch 8/50
1